<a href="https://colab.research.google.com/github/neuqrui/-GRPO-/blob/main/%E5%BC%B1%E6%99%BA%E5%90%A7%E5%BE%AE%E8%B0%83GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

知乎教程https://zhuanlan.zhihu.com/p/23035781247，
数据集采用hugingface ruozhiba数据集

**第1步**，下载必要的库（在本地运行和在colab上运行有些许差别的哦）

In [ ]:
# 跳过Colab的重启提示
import sys; modules = list(sys.modules.keys())
for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

!pip install unsloth vllm    # 安装核心库unsloth（加速训练）和vllm（快速推理）
!pip install --upgrade pillow  # 升级图像处理库
# 如果是本地运行，需要安装diffusers
# !pip install diffusers
# 安装特定版本的TRL库（支持GRPO算法）
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b
!pip install sentence-transformers #文本相似度所需的库

**第2步**，初始化Unsloth和GRPO

In [3]:
from unsloth import FastLanguageModel, PatchFastRL
PatchFastRL("GRPO", FastLanguageModel)  # 为GRPO算法打补丁以加速训练

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


**第3步**，加载预训练大模型，我这里使用Qwen2.5，使用lora进行微调

In [ ]:
from unsloth import is_bfloat16_supported
import torch

max_seq_length = 1024   # 模型支持的最大序列长度
lora_rank = 64         # LoRA的秩，值越大模型能力越强但速度越慢

# 从HuggingFace加载Qwen2.5-3B-Instruct模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-3B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True,       # 4位量化加载以节省显存
    fast_inference = True,     # 启用vLLM加速推理
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.5,  # GPU显存利用率（降低可缓解OOM）
)

# 为模型添加LoRA适配器
model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,            # LoRA秩
    target_modules = [         # 应用LoRA的目标模块
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,    # LoRA缩放系数
    use_gradient_checkpointing = "unsloth",  # 启用梯度检查点以支持长序列
    random_state = 666,       # 随机种子
)

测试预训练模型的输出

In [5]:
text = "请写一首春天的诗："
inputs = tokenizer(text, return_tensors='pt').to('cuda')

In [6]:
# outputs = model(**inputs)
FastLanguageModel.for_inference(model)
outputs = model.generate(
    input_ids = inputs['input_ids'], # 显式传递 input_ids
    attention_mask = inputs['attention_mask'], # **显式传递 attention_mask**
    max_length=100,      # 设置最大生成长度 (包括 prompt 和生成的文本)
    num_beams=1,         # **禁用集束搜索，设置 num_beams=1**
    temperature=0.7,     # 采样温度，控制生成文本的随机性 (可选，值越低越确定，越高越随机)
    top_k=50,            # Top-k 采样，限制候选词的范围 (可选)
    top_p=0.95,          # Top-p 采样 (nucleus sampling)，另一种限制候选词的方法 (可选)
    repetition_penalty=1.2, # 重复惩罚，减少生成重复文本的可能性 (可选)
    no_repeat_ngram_size=3, #  防止生成 n-gram 重复 (可选，例如防止 3-gram 重复)
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Prompt:\n", inputs)
print("\nGenerated Poem:\n", generated_text)

Prompt:
 {'input_ids': tensor([[ 14880,  61443, 108462, 105303,   9370, 100045,   5122]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Generated Poem:
 请写一首春天的诗： 春天，你来了
万物复苏，春意浓，
绿叶嫩芽破土而出。
鸟儿欢歌在枝头，
花香四溢，蜜蜂忙碌。

春风轻拂过田野，
麦苗青翠，生机勃勃。
小溪潺潺流淌着，
鱼儿嬉戏，欢快无比。

孩子们穿着新衣裳，
追逐蝴蝶，在草地上奔跑。
老人们坐在树荫下，
享受阳光，谈笑风


**第4步**，定义回答格式

In [7]:
SYSTEM_PROMPT = """
请使用中文按以下格式回答问题:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

XML_COT_FORMAT = """\
<reasoning>
{reasoning}
</reasoning>
<answer>
{answer}
</answer>
"""

**第5步**，hugingface弱智吧-数据加载及其预处理https://huggingface.co/datasets/LooksJuicy/ruozhiba

In [8]:
import re
from datasets import load_dataset, Dataset
import json

def load_custom_dataset(ds) -> Dataset:
    # 处理数据为符合训练格式
    processed_data = []
    for item in ds['train']:
      print(item)
      instruction = item["instruction"]
      output = item["output"]

      # 设定 prompt 格式（符合 chat 训练格式）
      prompt = [
          {"role": "system", "content": SYSTEM_PROMPT},
          {"role": "user", "content": instruction}
      ]

      processed_data.append({"prompt": prompt, "answer": output})

    # 转换为 Hugging Face Dataset
    dataset = Dataset.from_list(processed_data)
    return dataset


In [ ]:
ds = load_dataset("LooksJuicy/ruozhiba")
print(ds)
# 加载数据集
dataset = load_custom_dataset(ds)
dataset

**第6步**，定义各种奖励函数，我们使用GRPO这个强化学习算法，自然要设置一下reward。因为我们希望模型推理弱智吧这个任务，属于一个开放问答的任务，不像做数学题一样有一个死板的答案，因此我们需要基于语义的相似度给予不同的奖励。此外，还需要让模型保持一定的回答格式，如果回答格式正确，也奖励一朵小红花。

In [28]:
import re
from sentence_transformers import SentenceTransformer, util

#加载 Sentence Transformers 模型
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')

text1, text2 = '我喜欢猫','我很喜欢狗'

similaritie = util.cos_sim(semantic_model.encode(text1), semantic_model.encode(text2))
print(semantic_model.encode(text2).shape)
print(similaritie)

(384,)
tensor([[0.9953]])


In [ ]:
#语义相似度奖励
def semantic_similarity_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    responses = [completion[0]['content'].strip() for completion in completions]
    answer = [a.strip() for a in answer]

    # 计算相似度
    similarities = util.cos_sim(semantic_model.encode(responses), semantic_model.encode(answer))
    print(similaritie.shape)
    rewards = []
    for sim in similarities.diagonal().tolist():  # 取对角线上的值（单个样本的相似度）
        if sim > 0.9:
            rewards.append(2.0)  # 非常接近
        elif sim > 0.7:
            rewards.append(1.5)  # 相关性较高
        elif sim > 0.5:
            rewards.append(1.0)  # 可能部分正确
        else:
            rewards.append(0.0)  # 相关性低

    return rewards

# 严格格式奖励：必须完全匹配 <reasoning>...</reasoning><answer>...</answer>
def strict_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"^<reasoning>\n.*?\n</reasoning>\n<answer>\n.*?\n</answer>\n$"
    responses = [completion[0]["content"] for completion in completions]
    return [0.5 if re.match(pattern, r) else 0.0 for r in responses]

# 软格式奖励：只需包含 <reasoning> 和 <answer> 部分
def soft_format_reward_func(completions, **kwargs) -> list[float]:
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [completion[0]["content"] for completion in completions]
    return [0.5 if re.search(pattern, r) else 0.0 for r in responses]

def count_xml(text) -> float:
    count = 0.0
    if text.count("<reasoning>\n") == 1:
        count += 0.125
    if text.count("\n</reasoning>\n") == 1:
        count += 0.125
    if text.count("\n<answer>\n") == 1:
        count += 0.125
        count -= len(text.split("\n</answer>\n")[-1])*0.001
    if text.count("\n</answer>") == 1:
        count += 0.125
        count -= (len(text.split("\n</answer>")[-1]) - 1)*0.001
    return count

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    contents = [completion[0]["content"] for completion in completions]
    return [count_xml(c) for c in contents]

**第7步：**配置GRPO参数

In [1]:
from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    use_vllm = True,   # 使用vLLM加速推理
    learning_rate = 5e-6, # 学习率
    adam_beta1 = 0.9,   # Adam优化器参数
    adam_beta2 = 0.99,
    weight_decay = 0.1,  # 权重衰减
    warmup_ratio = 0.1,  # 学习率预热比例
    lr_scheduler_type = "cosine",  # 学习率调度策略
    optim = "adamw_8bit",      # 8位Adam优化器
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),   # 根据硬件支持选择精度
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 1,  # batch size,你计算资源够的话，可以设置高一点
    gradient_accumulation_steps = 1, # 累计1步后更新一次参数，时间换空间，将batch_size分为steps个mini_batch进行梯度累积，再更新参数
    num_generations = 8,  # 每次生成的候选数
    max_prompt_length = 256,  # 输入最大长度
    max_completion_length = 200,  # 生成最大长度

    max_steps = 200,    # 最大训练步数
    save_steps = 50,   # 保存间隔
    max_grad_norm = 0.1,   # 梯度裁剪阈值
    report_to = "none",
    output_dir = "outputs",
)

ModuleNotFoundError: No module named 'trl'

**第8步**，配置训练器，开始训练

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [   # 奖励函数列表
        xmlcount_reward_func,   # XML结构奖励
        soft_format_reward_func,  # 宽松格式奖励
        strict_format_reward_func,   # 严格格式奖励
        semantic_similarity_reward_func  # 语义相似奖励
    ],
    args = training_args,
    train_dataset = dataset,
)
trainer.train() #启动训练

**第9步**，保存lora

In [ ]:
model.save_lora("grpo_saved_lora")

**第10步**，使用训练好的模型，测试若至发言，看看他的推理过程。

In [1]:
from vllm import SamplingParams
def test_ruozhi(prompt):
  text = tokenizer.apply_chat_template([
    {"role" : "system", "content" : SYSTEM_PROMPT},
    {"role" : "user", "content" : prompt},
  ], tokenize = False, add_generation_prompt = True)


  sampling_params = SamplingParams(
    temperature = 0.8,  # 生成温度（越高越随机）
    top_p = 0.95,   # 核采样阈值
    max_tokens = 1024, # 最大生成token数
  )
  output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
  )[0].outputs[0].text
  print("\n")
  print(output)

ModuleNotFoundError: No module named 'vllm'

In [ ]:
test_ruozhi("我把漂白水喝了我的皮肤会不会变白？")